# Collecting the data - About the data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Superstore sales data.csv', encoding='windows-1252') 

In [2]:
# read all rows and columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# number of rows and columns of dataset
df.shape

In [ ]:
# columns of dataset
df.columns

# Data pre-processing

In [5]:
# read all columns
pd.set_option('display.max_columns', None)

In [ ]:
# data type of each column and null values (no null, need to change Date dtype)
df.info()

In [7]:
# change datatype for Order Date and Ship Date
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Ship Date'] = pd.to_datetime(df['Ship Date'])

In [ ]:
# check for duplicate values (no)
df.duplicated().value_counts()

In [ ]:
# count the unique values in each column
df.nunique()

In [10]:
# drop columns
df.drop(columns = ['Country', 'Postal Code'], inplace=True)

In [11]:
# split Order Date column
df['Order Year'] = df['Order Date'].astype(str).str.split('-', expand=True)[0]
df['Order Month'] = df['Order Date'].astype(str).str.split('-', expand=True)[1]

In [12]:
# delivery time
df['Delivery Time'] = (df['Ship Date'] - df['Order Date']) / np.timedelta64(1, 'D')

In [ ]:
df.sample(3)

In [14]:
# export the final file
df.to_csv('/content/drive/MyDrive/Colab Notebooks/Superstore sales data_cleaned.csv', index=False)

# EDA

Basic stats

In [ ]:
# basic stats of the dataset
df.describe()

In [ ]:
# correlation of the columns 
df.corr()
ax = sns.heatmap(df.corr(), cmap='coolwarm', annot=True)
plt.gcf().set_size_inches(8,5)

Sales revenue and profit overview

In [17]:
# total revenue 
total_sales = df['Sales'].sum()

In [18]:
# total profit
total_profit = df['Profit'].sum()

In [ ]:
# overall profit margin
total_profit / total_sales * 100

Product sales analysis

In [ ]:
# quantity per product
df.groupby(['Category', 'Sub-Category'])['Quantity'].sum()

In [ ]:
# sales by category
df.groupby(['Category'])['Sales'].sum()

In [ ]:
# profit by category 
df.groupby(['Category'])['Profit'].sum()

In [ ]:
# profit margin by sub-category
pm_product = df.groupby(['Category', 'Sub-Category'])[['Sales', 'Profit']].sum()
pm_product['Profit Margin'] = pm_product['Profit'] / pm_product['Sales'] * 100
pm_product.sort_values(by='Profit Margin', ascending=False)

In [24]:
# selling products (80-23)
total_sales_of_sp = df.groupby(['Category', 'Sub-Category', 'Product Name'])['Sales'].sum().sort_values(ascending=False).head(420).sum()

In [ ]:
# top 10 selling products
sp = df.groupby(['Sub-Category', 'Product Name'])['Sales'].sum().sort_values(ascending=False).head(10)
sp

In [26]:
# dead products (5-51)
total_sales_of_dp = df.groupby(['Sub-Category', 'Product Name'])['Sales'].sum().sort_values(ascending=False).tail(950).sum()

In [ ]:
# top 10 dead products
dp = df.groupby(['Sub-Category', 'Product Name'])['Sales'].sum().sort_values().head(10)
dp

Customers analysis

In [ ]:
# sales and profit by segment of customers and category
df.groupby(['Segment', 'Category'])[['Sales', 'Profit']].sum().sort_values(by = 'Sales', ascending=False)

In [ ]:
# profit margin by segment
pm_segment = df.groupby(['Segment'])[['Sales', 'Profit']].sum()
pm_segment['Profit Margin'] = pm_segment['Profit'] / pm_segment['Sales'] * 100
pm_segment

In [ ]:
# Pareto for sales by customers 
df1 = df.groupby(['Customer ID']).agg({'Sales':'sum'})
df1 = df1.sort_values(by = 'Sales', ascending=False)
n_customers = int(round(len(df1)*0.2))
top_20_percent_sales = df1['Sales'].head(n_customers).sum() / df1['Sales'].sum() * 100

print(f'The top {n_customers} customers generate {top_20_percent_sales:.2f}% of sales')

In [ ]:
# customers who bring about major sales (50-80) --> check Pareto
df.groupby(['Customer ID', 'Customer Name'])['Sales'].sum().sort_values(ascending=False).head(400).sum()

In [ ]:
# Pareto for profit by customers 
df2 = df.groupby(['Customer ID']).agg({'Profit':'sum'})
df2 = df2.sort_values(by = 'Profit', ascending=False)
n_customers = int(round(len(df2)*0.2))
top_20_percent = df2['Profit'].head(n_customers).sum() / df2['Profit'].sum() * 100

print(f'The top {n_customers} customers generate {top_20_percent:.2f}% of profit')

In [ ]:
# customers who bring about major profit (20-80) --> check Pareto
dg = df.groupby(['Customer ID', 'Customer Name']).agg({'Profit':'sum'}).sort_values(by = 'Profit', ascending=False)
dg.query('Profit > 0').head(155).sum()

In [ ]:
# customers' preference on ship mode 
df.groupby('Ship Mode')['Order ID'].count()

In [ ]:
# delivery time stats
df.groupby('Ship Mode')['Delivery Time'].describe()

In [ ]:
# sales and profit by delivery time 
df.groupby('Delivery Time')[['Sales', 'Profit']].sum()

Region analysis

In [ ]:
# overall sales and profit by states
df.groupby(['Region', 'State'])[['Sales', 'Profit']].sum().sort_values(by = 'Sales', ascending=False).head(16)

In [ ]:
# top 10 profitable states
df.groupby(['Region', 'State'])['Profit'].sum().sort_values(ascending=False).head(10)

In [ ]:
# top 10 loss states
df.groupby(['Region', 'State'])['Profit'].sum().sort_values(ascending=False).tail(10)

Sales and discount trend analysis

In [ ]:
# sales by year 
df.groupby('Order Year')['Sales'].sum().sort_values(ascending=False)

In [ ]:
# sales by month
df.groupby('Order Month')['Sales'].sum().sort_values(ascending=False)

In [ ]:
df['Sales'].sum()

In [ ]:
# profit by year
df.groupby('Order Year')['Profit'].sum().sort_values(ascending=False)

In [ ]:
# profit by month
df.groupby('Order Month')['Profit'].sum().sort_values(ascending=False)

In [ ]:
# profit margin by year
pm_year = df.groupby(['Order Year'])[['Sales', 'Profit']].sum()
pm_year['Profit Margin'] = pm_year['Profit'] / pm_year['Sales'] * 100
pm_year

In [ ]:
# discount by year 
df.groupby(['Order Year', 'Category'])['Discount'].mean().sort_values(ascending=False)

In [ ]:
# discount by month
df.groupby(['Order Month', 'Category'])['Discount'].mean().sort_values(ascending=False).head(10)